<a href="https://colab.research.google.com/github/noorelhaj98-ship-it/pytorch-tutorial/blob/main/Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
import torch


In [ ]:
m_name = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(m_name)
tokenizer.pad_token = tokenizer.eos_token  # Fix padding issue

model = AutoModelForCausalLM.from_pretrained(m_name)
model.config.pad_token_id = tokenizer.pad_token_id


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
texts = [
    "Roses are red, violets are blue, machine learning is fun and so are you.",
    "Robots can dance when their code says so, spinning around like a perfect show.",
    "When computers dream, they think in code, exploring ideas in a digital mode."
]

dataset = Dataset.from_dict({"text": texts})
dataset


Dataset({
    features: ['text'],
    num_rows: 3
})

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True)

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
def tokenize_and_add_labels(batch):
    tokens = tokenizer(batch["text"], truncation=True)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_and_add_labels, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")



Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

# Training arguments
training_args = TrainingArguments(
    output_dir="tiny_finetuned_model",
    num_train_epochs=1,             # quick training
    per_device_train_batch_size=1,
    logging_steps=1,
    save_total_limit=1,
    report_to="none"                # <- disables W&B / any other logging
)

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

# Train the model
trainer.train()


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,4.442900
2,5.152400
3,5.464400


TrainOutput(global_step=3, training_loss=5.019893646240234, metrics={'train_runtime': 19.1958, 'train_samples_per_second': 0.156, 'train_steps_per_second': 0.156, 'total_flos': 13524148224.0, 'train_loss': 5.019893646240234, 'epoch': 1.0})

In [ ]:
def generate(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(inputs["input_ids"], max_length=50)
    return tokenizer.decode(output[0], skip_special_tokens=True)


before Fine Tuning

In [ ]:
original_model = AutoModelForCausalLM.from_pretrained("distilgpt2")
print("Before fine-tuning:\n")
print(generate("Write a short poem:", original_model, tokenizer))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Before fine-tuning:

Write a short poem:















































After Fine tuning

In [ ]:
print("\nAfter fine-tuning:\n")
print(generate("Write a short poem:", model, tokenizer))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



After fine-tuning:

Write a short poem:















































Behavior Changes After Fine-Tuning

Before fine-tuning, DistilGPT-2 generated random text that did not follow a pattern
After fine-tuning on my small poem dataset, the model started producing output that was more poetic and closer to the training examples. It used rhyme, simple structure, and soft tone like the dataset poems.

This shows that even a small dataset can change the writing style and behavior of a language model.